### CONFIGURATION

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn

In [3]:
#CHROM SLICING
minChromFrac = 0
maxChromFrac = 26

In [13]:
# BED FILE
BED_PATH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/70-30mixin/"
BED_FILE_NAME = "na12878-na24385-somatic-truth-regionsSORTED.bed"

# TRUTH SET
SAMPLE_NAMES_TRUTH = {'NA12878':'70-30truth'}
VCF_PATH_TRUTH = "/Users/peterpriestley/hmf/analyses/giabTruthsets/70-30mixin/"
VCF_FILE_NAME_TRUTH = "na12878-na24385-somatic-truth.vcf"

# COMBINED VCF CONFIG
VCF_SAMPLE = "CPCT11111111"
VCF_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/161213_GIAB_mixin_reference_run_v1.11/"
VCF_FILE_NAME = VCF_SAMPLE + "R_"+ VCF_SAMPLE + "T_merged_somatics.vcf"#_somatic_filtered.vcf"#
SAMPLE_NAMES = {VCF_SAMPLE + 'T.mutect':'mutect', \
                VCF_SAMPLE + 'T.freebayes':'freebayes', \
                'TUMOR.strelka':'strelka', \
                'TUMOR.varscan':'varscan'}
#SAMPLE_NAMES = {VCF_SAMPLE + 'T':'freebayes',VCF_SAMPLE + 'R':'normal'}

### Functions

In [5]:
def filterByChromFrac(df):
    return df[(df.chromFrac > minChromFrac)&(df.chromFrac < maxChromFrac)]

In [6]:
def calculateTruth(df,dfTruth):
    df = pd.merge(df,dfTruth,how='left', left_index=True,right_index=True,suffixes=('', '_Truth'))
    df['hasTP'] = False
    df['hasFP'] = False
    for columnName in list(df):
        if columnName.endswith('allele') and not columnName.startswith('truth'):
            df['hasTP'] = (df['hasTP']) | ((df[columnName[:-6]+'indelDiff'] == df['truth_indelDiff']) \
                    & (~pd.isnull(df['truth_indelDiff']) & (df['variantType'] == 'INDEL'))) |((df[columnName] == df['truth_allele']) \
                    & (df['variantType'] == 'SNP'))
            df['hasFP'] = (df['hasFP']) | ((df[columnName[:-6]+'indelDiff'] != df['truth_indelDiff']) \
                    & (df['variantType'] == 'INDEL') & (df[columnName[:-6]+'indelDiff'] != '')& (~pd.isnull(df['truth_allele']))) |((df[columnName] != df['truth_allele']) \
                    & (df['variantType'] == 'SNP') & (df[columnName] != '')& (~pd.isnull(df['truth_allele'])))
    df['Truth'] = (df['hasTP']) &  (df['hasFP'] == False)
    return df

In [7]:
def calcuatePrecisionSensivityMatrix(df):
    outputdata = []
    for columnName in list(df):
        if columnName.endswith('allele') & ~columnName.endswith('_allele'):
            myCaller = columnName[:-6]
            variantTypes = df[(df[myCaller+'allele'] != '')].variantType.unique()
            for variantType in variantTypes:
                truePositives = len(df[(df[myCaller+'allele'] != '') & (df['Truth'] == True) &(df['variantType'] == variantType)])
                positives = len(df[(df[myCaller+'allele'] != '')&(df['variantType'] == variantType)])
                truthSet = len(dfTruth[dfTruth['variantType'] == variantType]) 
                falseNegatives = truthSet - truePositives
                if positives > 0 and truthSet > 0:
                    outputdata.append([variantType, myCaller, truthSet,truePositives,positives-truePositives, falseNegatives, \
                                   round(truePositives/float(positives),4),round(truePositives/float(truthSet),4)])
    
    outputDF = pd.DataFrame(outputdata)
    outputDF.columns = (['variantType','caller','truthSet','truePositives','falsePositives','falseNegatives','precision','sensitivity'])
    return outputDF.sort_values(['variantType','caller'])

<h3> Load VCFs and Prepare DF

In [14]:
## LOAD BED
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)

In [15]:
## LOAD TRUTH SET VCF
truthBed = copy.deepcopy(bed)
dfTruth = aVCF.loadVaraintsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH, \
                                   SAMPLE_NAMES_TRUTH,"Mix-in Truth Set",True,True,truthBed)
dfTruth = filterByChromFrac(dfTruth)
dfTruth.rename(columns={'70-30truthallele':'truth_allele','70-30truthindelDiff':'truth_indelDiff'}, inplace=True)
dfTruth = dfTruth[['chrom','pos','variantType','ref','truth_allele','truth_indelDiff','bedRegion']]
dfTruth = dfTruth.set_index(['chrom','pos'])

reading vcf file: na12878-na24385-somatic-truth.vcf
reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
reading VCF File line: 1100000
Number variants loaded: 1104460


In [16]:
# LOAD SAMPLE VCF + match to truth set
sampleBed = copy.deepcopy(bed)
dfProd = aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,VCF_SAMPLE,True,True,sampleBed)
dfProd = filterByChromFrac(dfProd)
dfProd = dfProd.set_index(['chrom','pos'])
dfProd = calculateTruth(dfProd,dfTruth)

reading vcf file: CPCT11111111R_CPCT11111111T_merged_somatics.vcf
reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
reading VCF File line: 1100000
reading VCF File line: 1200000
reading VCF File line: 1300000
reading VCF File line: 1400000
Number variants loaded: 1079934


### PRECISION + SENSITIVITY|

In [17]:
outputDF = calcuatePrecisionSensivityMatrix(dfProd)
outputDF

,variantType,caller,truthSet,truePositives,falsePositives,falseNegatives,precision,sensitivity
6,INDEL,freebayes,96616,64343,692,32273,0.9894,0.6660
1,INDEL,strelka,96616,48365,273,48251,0.9944,0.5006
3,INDEL,varscan,96616,63570,623,33046,0.9903,0.6580
5,SNP,freebayes,1007844,935473,1006,72371,0.9989,0.9282
4,SNP,mutect,1007844,931973,5947,75871,0.9937,0.9247
0,SNP,strelka,1007844,941410,3525,66434,0.9963,0.9341
2,SNP,varscan,1007844,899598,832,108246,0.9991,0.8926


In [18]:
dftemp = dfProd.reset_index()
pd.pivot_table(dftemp[(dftemp.variantType=='INDEL')|(dftemp.variantType=='SNP')], values='pos', index=['numCallers'], columns=['variantType','Truth'], aggfunc='count')

variantType   INDEL              SNP          
Truth         False    True    False     True 
numCallers                                    
1            1094.0  12311.0  7485.0   23870.0
2             205.0  17884.0  1586.0   56804.0
3              28.0  42733.0   147.0   91816.0
4               NaN      NaN    53.0  823882.0

### SCRATCH

In [25]:
pd.pivot_table(dftemp[((dftemp.variantType=='INDEL')|(dftemp.variantType=='SNP'))&(dftemp.strelkaAF > 0.1)], values='pos', index=['numCallers','vennSegment'], columns=['variantType','Truth'], aggfunc='count')

variantType                           INDEL              SNP          
Truth                                 False    True    False     True 
numCallers vennSegment                                                
1          freebayes                  475.0   5995.0   657.0    6872.0
           mutect                       NaN      NaN  4383.0    9353.0
           strelka                     50.0   1031.0   276.0    1668.0
           varscan                    418.0   4894.0   513.0    2871.0
2          freebayes-mutect             NaN      NaN    41.0   11305.0
           strelka-freebayes           32.0   1941.0    48.0    3539.0
           strelka-mutect               NaN      NaN    67.0   11554.0
           strelka-varscan             16.0   2221.0    33.0    1097.0
           varscan-freebayes          157.0  13613.0    67.0    8239.0
           varscan-mutect               NaN      NaN    21.0     779.0
3          strelka-freebayes-mutect     NaN      NaN    41.0   28179.0
           strelka-varscan-freebayes   28.0  42630.0    65.0   35540.0
           strelka-varscan-mutect       NaN      NaN    10.0    7266.0
           varscan-freebayes-mutect     NaN      NaN    13.0   15424.0
4          Intersection                 NaN      NaN    51.0  819304.0

In [18]:
pd.pivot_table(dftemp[((dftemp.variantType=='INDEL')|(dftemp.variantType=='SNP'))&(dftemp.strelkaAF < 0.1)], values='pos', index=['numCallers','vennSegment'], columns=['variantType','Truth'], aggfunc='count')

variantType                           INDEL            SNP         
Truth                                 False  True    False    True 
numCallers vennSegment                                             
1          strelka                    146.0  359.0  1582.0   2956.0
2          strelka-freebayes            NaN    6.0     7.0    201.0
           strelka-mutect               NaN    NaN  1257.0  18735.0
           strelka-varscan              NaN   40.0    35.0    427.0
3          strelka-freebayes-mutect     NaN    NaN     3.0   1139.0
           strelka-varscan-freebayes    NaN    6.0     1.0    657.0
           strelka-varscan-mutect       NaN    NaN    14.0   2848.0
4          Intersection                 NaN    NaN     1.0   2186.0

In [ ]:
dftemp[dftemp.Truth == False].head(10)

<h3> SNP

In [ ]:
snpdf = dfProd[(dfProd.variantType == 'SNP')]

In [ ]:
vn.venn([snpdf[snpdf.mutectallele != '']['chromPos'], \
         snpdf[snpdf.strelkaallele != '']['chromPos'], \
        snpdf[snpdf.freebayesallele != '']['chromPos'], \
        snpdf[snpdf.varscanallele != '']['chromPos'] \
        ],['mutect','strelka','freebayes','varscan'],figsize=(6,6))

### <hd3> Indel Venn

In [ ]:
indeldf= dfProd[(dfProd.variantType == 'INDEL')]

In [ ]:
vn.venn([indeldf[indeldf.strelkaallele != '']['chromPos'], \
        indeldf[indeldf.freebayesallele != '']['chromPos'], \
        indeldf[indeldf.varscanallele != '']['chromPos'] \
        ],['strelka','freebayes','varscan'],figsize=(6,6))

<h3> Allelic Depth

In [ ]:
#Alllele Freq By Caller
for columnName in list(dfProd):
    if columnName.endswith('freebayesAF'):
        print columnName
        ser = dfProd[(dfProd.Truth == True)][columnName[:-2] + 'AF']
        ser = ser.sort_values()
        #ser[len(ser)] = ser.iloc[-1]
        cum_dist = np.linspace(0.,1.,len(ser))
        ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+" m="+str(round(ser.median(),2)))
        ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[0,1])

In [ ]:
truths = [True,False]
for truth in truths:
    #Alllele Freq By Caller
    for columnName in list(dfProd):
        if columnName.endswith('freebayesQS') and not columnName.endswith('lQS'):
            ser = dfProd[(dfProd.Truth == truth)][columnName]
            ser = ser.sort_values()
            cum_dist = np.linspace(0.,1.,len(ser))
            ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+ " "+ str(truth) +" m="+str(round(ser.median(),2)))
            ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[0,100],ylim=[0,0.5])

In [ ]:
truths = [True,False]
for truth in truths:
    #Alllele Freq By Caller
    for columnName in list(dfProd):
        if columnName.endswith('freebayesQS') and not columnName.endswith('lQS'):
            ser = dfProd[(dfProd.Truth == truth)][columnName]
            ser = ser.sort_values()
            cum_dist = np.linspace(0.,1.,len(ser))
            ser_cdf = pd.Series(cum_dist, index=ser,name=columnName[:-2]+": c="+str(ser.count())+ " "+ str(truth) +" m="+str(round(ser.median(),2)))
            ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller (AllelicFreq > 0.0)",figsize=[15,6],xlim=[-20,0],ylim=[0.6,1])

In [ ]:
ser

In [ ]:
ser.head()